Build a Crew to Tailor Job Applications

In [22]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [23]:
from crewai import Agent, Task, Crew

In [24]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

crewAI Tools

In [31]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./cv_seo.md')
semantic_search_resume = MDXSearchTool(mdx='./cv_seo.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.80it/s]


In [ ]:
#Uncomment and run the cell below if you wish to view fake_resume.md in the notebook

# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

Creating Agents

In [32]:
# Agent 1: Researcher
researcher = Agent(
    role="SEO Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [33]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for SEO experts",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [34]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for SEO experts",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [35]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="SEO experts Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

Creating Tasks

In [9]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [36]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using my Linkedin profile ({linkedin_profile_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [37]:
#You can pass a list of tasks as context to a task.
#The task then takes into account the output of those tasks in its execution.
#The task will not run until it has the output(s) from those tasks

# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [38]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


Creating the Crew

In [39]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

2024-07-01 12:27:34,534 - 55444 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


Running the Crew

In [40]:
job_application_inputs = {
    'job_posting_url': 'https://www.linkedin.com/jobs/view/3952546138/?refId=7c1d62a0-682e-3643-8db3-f0549f53f5d2',
    'linkedin_profile_url': 'https://www.linkedin.com/in/gabinoguera/',
    'personal_writeup': """Hi, I am Gabriel, currently IT Project Manager at Educa Edtech Group of the
Euroinnova Group. My main career has been linked to digital marketing and SEO but
in the last few years I have specialized in artificial intelligence and programming,
mainly with Python.
I am currently finishing a Master in artificial intelligence and knowledge engineering
that includes knowledge and management of data mining and visualization tools
such as Hadoop or Power BI, among others.
Additionally I have developed AI pipelines using Langchain, GPT, Tensorflow, NLP,
Hugging Face, etc. mainly for applications related to content generation and
analytics of trends, traffic, audience and end-user understanding to improve
recommendation systems, advertising and personalization.
I have worked in both fast-paced and multitask startup environments as well as in
larger companies like the current one, where there are more solid and specialized
teamwork structures with agile methodologies"""
}

In [41]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://www.linkedin.com/jobs/view/3952546138/?refId=7c1d62a0-682e-3643-8db3-f0549f53f5d2) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for SEO experts
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using my Linkedin profile (https://www.linkedin.com/in/gabinoguera/) URLs, and personal write-up (Hi, I am Gabriel, currently IT Project Manager at Educa Edtech Group of the
Euroinnova Group. My main career has been linked to digital marketing and SEO but
in the last few years I have specialized in artificial intelligence and programming,
mainly with Python.
I am currently finishing a Master in artificial intelligence and knowledge engineering
that include

In [42]:
#Dislplay the generated tailored_resume.md file.

from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

**Gabriel Guera**
**Contact Information**
- Email: gabriel.guera@example.com
- Phone: +1234567890
- LinkedIn: linkedin.com/in/gabrielguera

**Professional Summary**
Accomplished IT Project Manager and SEO Strategist with over 4 years of experience in digital marketing, specializing in artificial intelligence applications for SEO. Proven track record in developing AI-driven SEO strategies, optimizing content for voice and visual search, and enhancing user experience through technical SEO. Skilled in collaborative leadership, capable of integrating cross-functional teams to achieve business objectives and drive growth.

**Professional Experience**
**Senior IT Project Manager & SEO Strategist**
Educa Edtech Group, Euroinnova Group
- Developed and implemented SEO strategies that increased organic traffic by 30% within one year.
- Led AI initiatives using platforms like Langchain, GPT, and TensorFlow to automate and enhance SEO processes.
- Optimized content for voice and visual search, increasing engagement and visibility across platforms.
- Conducted technical SEO audits, improving Core Web Vitals and site performance metrics.
- Collaborated with marketing and technical teams to integrate SEO with overall business strategies.

**Education**
Master in Artificial Intelligence and Knowledge Engineering (Expected 2023)
University of Technology

Bachelor of Science in Digital Marketing
University of Marketing Sciences

**Skills**
- SEO Tools: Semrush, Ahrefs, Screaming Frog, Google Search Console
- Programming: Python, JavaScript, SQL
- AI and Machine Learning: Langchain, GPT, TensorFlow, NLP, Hugging Face
- Technical SEO: Web Performance Optimization, Structured Data, Core Web Vitals
- Content Strategy: Keyword Research, AI Copywriting Tools, Content Optimization for Voice and Visual Search

**Certifications**
- Certified SEO Strategist, Digital Marketing Institute
- Google Analytics Certified

**Professional Achievements**
- Published a paper on "AI Enhancements in Search Engine Optimization" in the Journal of Digital Marketing.
- Speaker at the International SEO Conference 2022, discussing advances in AI-driven SEO strategies.

**Languages**
- English (Fluent)
- Spanish (Intermediate)

**Interests**
- Continuous learning in AI and SEO advancements
- Developing open-source tools for SEO automation

**References**
Available upon request.

By strategically incorporating the latest SEO trends and Gabriel's extensive experience in AI and technical skills, this resume is tailored to meet the specific requirements of the Senior SEO Strategist position at Fever, showcasing his suitability and readiness for the role.

In [43]:
# Dislplay the generated interview_materials.md file.

display(Markdown("./interview_materials.md"))

**Interview Preparation Document for Gabriel Guera - Senior SEO Strategist Position at Fever**

**Key Interview Questions:**

1. **SEO Strategy and Implementation:**
   - "Can you discuss a specific instance where you developed and implemented an SEO strategy that was particularly successful in meeting business objectives?"
   - "How do you tailor your SEO strategies to different business needs and objectives, as you did previously in sectors like education and ecommerce?"

2. **Technical SEO:**
   - "You've managed portal migrations and implemented technical enhancements like sitemaps and structured data. Can you explain the process and the impact of these changes on SEO performance?"
   - "What technical SEO challenges have you faced and how did you overcome them?"

3. **Content SEO:**
   - "Given your experience with content strategies tailored to buyer personas, how do you conduct keyword research to uncover new business and content opportunities?"
   - "Can you provide an example of how you optimized user intent for a specific project and its outcome?"

4. **Performance Monitoring:**
   - "How do you monitor and evaluate the effectiveness of your SEO strategies? Could you share tools and methodologies you use for this purpose?"
   - "Discuss a time when you had to adapt your strategy based on performance data or competitor benchmarks."

5. **Traffic and Conversion Analysis:**
   - "Can you describe a project where you conducted in-depth traffic and conversion analyses? What insights did you gather and how did they influence your SEO strategy?"

6. **International SEO:**
   - "You led the SEO strategy for a UK market entry. What specific considerations did you take into account for international SEO?"

7. **Collaboration:**
   - "How do you ensure effective collaboration between technical and marketing teams in your projects? Can you share an experience where this was particularly critical?"

**Talking Points for Gabriel to Highlight:**

- **Proven Track Record in SEO:**
  - Emphasize the successful SEO strategies and the tangible results achieved, such as the 30% increase in organic traffic at Educa Edtech Group.
  
- **Expertise in Technical SEO:**
  - Discuss the technical enhancements implemented and their direct benefits, such as improved site performance metrics and Core Web Vitals.
  
- **Leadership in AI-Driven SEO:**
  - Highlight the leadership in AI initiatives and the use of platforms like Langchain and TensorFlow to automate and enhance SEO processes.
  
- **Collaborative Skills:**
  - Showcase the ability to integrate cross-functional teams and align SEO strategies with overall business goals, as demonstrated in previous roles.
  
- **Continuous Learning and Adaptation:**
  - Mention ongoing education and the application of the latest technologies in AI and SEO, reinforcing a commitment to staying at the forefront of industry developments.

This document should help Gabriel prepare effectively for his interview, focusing on his strengths and aligning his experiences with the job requirements at Fever.